In [ ]:
import csv, json, sys, os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


## json

In [ ]:
fileInput = '/content/5471908.json'
inputFile = open(fileInput) #open json file
data = json.load(inputFile) #load json content
#json_df = pd.read_json(inputFile)
inputFile.close() #close the input file

In [ ]:
#fileOutput = '/content/5471908.csv'
#outputFile = open(fileOutput, 'w') #load csv file
#output = csv.writer(outputFile) #create a csv.write
#output.writerow(data[0].keys())  # header row
#for row in data:
#    output.writerow(row.values()) #values row

In [ ]:
datalist = list(data.values())
for key, item in zip(data.keys(), datalist):
    print(key,":",item)

In [ ]:
steps = data['steps']
steps[len(steps) - 1]

In [ ]:
s = steps[len(steps) - 1]
s[0]['observation']['board']

In [ ]:
player_actions = []
opp_actions =  []
board_list = []
for s in steps:
    player_actions += [ s[0]['action'] ]
    opp_actions += [ s[1]['action'] ]
    board_list += [ s[0]['observation']['board'] ]
#player_actions, opp_actions, 
board_list

In [ ]:
df = pd.DataFrame(board_list)
df["play"] = player_actions
df["opp"] = opp_actions
df

In [ ]:
board -= 1
print ("Board:",board)
print ("Player played:", df["play"][board])
print ("Opponent played:", df["opp"][board])
grid = np.array(df.iloc[board])#
plt.imshow(grid[:-2].reshape((6,7)))

##csv

###pandas

In [ ]:
good_moves_df = pd.read_csv('/content/good_move_archive.csv', sep=';', header=None)
good_moves_df = good_moves_df.drop(1,axis=1)
good_moves_series = good_moves_df[0]
good_moves_series

In [ ]:
good_moves_series.to_csv('/content/good_moves.csv', header=False)

In [ ]:
#del good_moves_df
good_moves_df = pd.read_csv('/content/good_moves.csv', index_col=0, header=None)
good_moves_df

In [ ]:
gm_arr = good_moves_df.values
gm = gm_arr[1]
gm

In [ ]:
gm_str = gm[0]
gm_str.split(sep='')
for x in gm_str:
    print(x)

###semi

In [ ]:
gmsemi_df = pd.read_csv('/content/good_move_semic.csv', sep=';', header=None)
gmsemi_df[0]

In [ ]:
boards = [b.split(": ")[1] for b in gmsemi_df[0]]
boards = [b.replace("[","").replace("]","") for b in boards]
boards = [b_str.split(", ") for b_str in boards]
for i, blist in enumerate(boards):
    boards[i] = [int(b) for b in blist]
boards = np.array(boards)
boards

In [ ]:
listlist = [list(b) for b in boards]
listlist

In [ ]:
moves = [m.split(": ")[1] for m in gmsemi_df[2]]
moves = [m.replace("[","").replace("]","") for m in moves]
moves = [m_str.split(", ") for m_str in moves]
for i, mlist in enumerate(moves):
    moves[i] = [int(m) for m in mlist]
moves = np.array(moves)
moves

In [ ]:
scores = [s.split(": ")[1] for s in gmsemi_df[1]]
scores = [s.replace("[","").replace("]","") for s in scores]
scores = [s.split(", ") for s in scores]
for i, slist in enumerate(scores):
    scores[i] = [int(s) for s in slist]
scores = np.array(scores)
scores = scores.flatten().tolist()
scores

In [179]:
pd.DataFrame(moves).to_csv("/content/moves.csv")

In [ ]:
import os
with open() as fh:    
    moves.tofile(fh)

In [ ]:
scores[11],list(moves[11]),list(boards[11])

In [ ]:
labels = [np.argmax(move) for move in moves] 
np.array(labels)

In [ ]:
training_set = pd.DataFrame(listlist)
#training_set["boards"] = boards
training_set["labels"] = labels
training_set.to_csv('training_set')

In [ ]:
del training_set
training_set = pd.read_csv('/content/training_set', index_col=0)
training_set